Luke Sidle - 10/11/2024
Difference between Video Game Music and Popular music

The data used in this report is all sourced from Spotify endpoints

My Hypothesis: Video Game Music and Popular music will have different mixing styles and stats. I think this is interesting theoretically because the differences between Video Game music and popular music could have implications for streaming statistics and why some people prefer Video Game Soundtracks to Popular Music. Statistically, this is interesting because seeing the differences between VGM and Popular music boiled down to the raw data is a unique way to understand the differences.

I will be using the endpoints, under audio features, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentless, liveness, tempo, and valence.

Here I import all the necessary starting programs. 

In [123]:
import urllib
import requests
import pandas as pd
import json
import base64

In [124]:
def get_session_token(SessionID, SessionKey):
    url = 'https://accounts.spotify.com/api/token'
    data = {'grant_type':'client_credentials'}
    encoded_key = base64.b64encode(str(SessionID + ":" + SessionKey).\
                                   encode("ascii"))
    header = {'Authorization':'Basic {}'.format(encoded_key.decode("ascii"))}
    response = requests.post(url, data = data, headers = header)
    print(response.status_code)
    return response.json()['access_token']

In [125]:
keys = pd.read_csv(r"C:\Users\datgu\OneDrive\Documents\spotifykeys.txt")

In [126]:
access_token = get_session_token(keys['Client_ID'].iloc[0], keys['Client_Secret'].iloc[0])

200


In [127]:
t_features_ep = 'https://api.spotify.com/v1/audio-features'
tracks_ep = 'https://api.spotify.com/v1/tracks'
ab_tracks_ep = 'https://api.spotify.com/v1/albums/{}/tracks'

In [128]:
def api_call(endpoint_url, api_header):
    response = requests.get(endpoint_url, headers = api_header)
    print(response.status_code)
    return response.json()

In [129]:
session_header = {'Authorization': 'Bearer {}'.format(access_token)}

Here I am choosing the different albums to use. For the popular music I chose the #1 album of the year streaming wise, Taylor Swift's "Tortured Poet's Department." For the Video Game soundtrack I am picking Undertale's since one of the it's songs, Megalovania, has the most streams out of any video game song.

In [130]:
ts_album_id = '1Mo4aZ8pdj6L1jx8zSwJnt'
ut_album_id = '2M2Ae2SvZe3fmzUtlVOV5Z'

In [131]:
ab_tracks_ep.format(ts_album_id)

'https://api.spotify.com/v1/albums/1Mo4aZ8pdj6L1jx8zSwJnt/tracks'

Making sure the formatting is working 

In [132]:
ts_album_response = api_call(ab_tracks_ep.format(ts_album_id), session_header)
ut_album_response = api_call(ab_tracks_ep.format(ut_album_id), session_header)

200
200


Using this to look at all the variables I can pull from.

In [133]:
ts_album_df = pd.DataFrame(ts_album_response['items'])
ut_album_df = pd.DataFrame(ut_album_response['items'])
ut_album_df.head()

,artists,available_markets,disc_number,duration_ms,explicit,external_urls,href,id,name,preview_url,track_number,type,uri,is_local
0,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,88966,False,{'spotify': 'https://open.spotify.com/track/4X...,https://api.spotify.com/v1/tracks/4XX5uZb9PvTK...,4XX5uZb9PvTKh8Nm2KSJfk,Once Upon a Time,https://p.scdn.co/mp3-preview/29c282b2b342eee4...,1,track,spotify:track:4XX5uZb9PvTKh8Nm2KSJfk,False
1,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,32000,False,{'spotify': 'https://open.spotify.com/track/5p...,https://api.spotify.com/v1/tracks/5pb39nk7Wr8e...,5pb39nk7Wr8e0aMQ6VUdlr,Start Menu,https://p.scdn.co/mp3-preview/19edba36fe1f2554...,2,track,spotify:track:5pb39nk7Wr8e0aMQ6VUdlr,False
2,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,23225,False,{'spotify': 'https://open.spotify.com/track/6g...,https://api.spotify.com/v1/tracks/6gJTgSwOw2Tq...,6gJTgSwOw2TqAPCnk2gTlk,Your Best Friend,https://p.scdn.co/mp3-preview/8ced1552fbbf4312...,3,track,spotify:track:6gJTgSwOw2TqAPCnk2gTlk,False
3,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,57818,False,{'spotify': 'https://open.spotify.com/track/1j...,https://api.spotify.com/v1/tracks/1jDMi92a9zNQ...,1jDMi92a9zNQuPD3uPMkla,Fallen Down,https://p.scdn.co/mp3-preview/141b3339b2910f8e...,4,track,spotify:track:1jDMi92a9zNQuPD3uPMkla,False
4,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,92282,False,{'spotify': 'https://open.spotify.com/track/1v...,https://api.spotify.com/v1/tracks/1vuSdk2EGjh3...,1vuSdk2EGjh3eXCXBbT9Qf,Ruins,https://p.scdn.co/mp3-preview/f6b802f444b692cd...,5,track,spotify:track:1vuSdk2EGjh3eXCXBbT9Qf,False


Using the join feature to add id to the album dataframe.

In [134]:
ts_track_ids = ','.join(ts_album_df['id'].to_list())
ut_track_ids = ','.join(ut_album_df['id'].to_list())

In [135]:
t_features_ep + '?ids={}'.format(ts_track_ids)

'https://api.spotify.com/v1/audio-features?ids=2OzhQlSqBEmt7hmkYxfT6m,3NMrVbIVWT3fPXBj0rNDKG,2XXwLdtuAcE0HSCu61ijAb,2F3N9tdombb64aW6VtZOdo,3Vevii7qKqrmW8CcyzBHDl,5og4Qzt92jJzVDkOtSEilb,3fO566xJgwxIa3qGCGBvIC,3ZVFcD8Wlw9T9klGqmJf9F,0W0iAC1VGlB82PI6elxFYf,36t6frENUtCYKuZus6aYDO,2h3MDMf8sd9s4XIzpTBIjX,6RSG1dKPV5gEvILwJb4QtS,4q5YezDOIPcoLr8R81x9qy,2v1ivOOsgn64g5OywuH55L,1dhMTsEZz6ZEquGvmzVoHn,1UlhrRvYzbHEyugEDspXUB'

Checking that the code is calling back to the api and condensing it into one command.

In [136]:
ts_track_features = api_call(t_features_ep + '?ids={}'.format(ts_track_ids), session_header)
ut_track_features = api_call(t_features_ep + '?ids={}'.format(ut_track_ids), session_header)

200
200


Checking that the code is calling back to the api, making sure we are using the right market, and condensing it into one command.


In [137]:
ts_track_info = api_call(tracks_ep + '?market=US&ids={}'.format(ts_track_ids), session_header)
ut_track_info = api_call(tracks_ep + '?market=US&ids={}'.format(ut_track_ids), session_header)

200
200


Creating a data frame for the audio features of the tracks.

In [138]:
ts_features_df = pd.DataFrame(ts_track_features['audio_features'])
ut_features_df = pd.DataFrame(ut_track_features['audio_features'])

Creating a dataframe for the tracks info outside of audio features.

In [139]:
ts_tracks_df = pd.DataFrame(ts_track_info['tracks'])
ut_tracks_df = pd.DataFrame(ut_track_info['tracks'])

Merging the two dataframe that were previously created in the last two steps

In [140]:
ts_merged = pd.merge(ts_features_df, ts_tracks_df, how = 'inner', on = 'id')
ut_merged = pd.merge(ut_features_df, ut_tracks_df, how = 'inner', on = 'id')

Notating which artist is which in the merged lists

In [141]:
ts_merged['artist'] = "Taylor Swift"
ut_merged['artist'] = "Undertale"

Creating a command to view the merged lists together.

In [142]:
artist_compare = pd.concat([ts_merged,ut_merged], axis = 0)

In this line of code I drop every variable that is not valuble to my hypothesis.

In [143]:
artist_compare_tidy = artist_compare.drop(columns = ['href', 'is_local', 'is_playable', 'track_number', 'type_y', 'uri_y', 'preview_url', 'album', 'artists', 'disc_number', 'analysis_url', 'duration_ms_x', 'uri_x', 'track_href', 'type_x', 'explicit', 'external_ids', 'type_x', 'id', 'external_urls'])

Above I display the tidy data combined in one list. 

In [144]:
artist_compare_tidy

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms_y,name,popularity,artist
0,0.504,0.38600,11,-10.976,1,0.0308,0.502000,0.000015,0.0961,0.2810,192.004,4,228965,Fortnight (feat. Post Malone),86,Taylor Swift
1,0.604,0.42800,0,-8.441,1,0.0255,0.048300,0.000000,0.1260,0.2920,110.259,4,293048,The Tortured Poets Department,71,Taylor Swift
2,0.596,0.56300,0,-7.362,1,0.0269,0.137000,0.000000,0.3020,0.4810,97.073,4,203801,My Boy Only Breaks His Favorite Toys,72,Taylor Swift
3,0.541,0.36600,11,-10.412,1,0.0748,0.560000,0.000001,0.0946,0.1680,159.707,4,261228,Down Bad,78,Taylor Swift
4,0.423,0.53300,9,-11.388,1,0.3220,0.730000,0.002640,0.0816,0.2480,160.218,4,262974,"So Long, London",72,Taylor Swift
5,0.521,0.72000,4,-7.684,1,0.1040,0.384000,0.000000,0.1350,0.4380,79.943,4,340428,But Daddy I Love Him,73,Taylor Swift
6,0.330,0.48300,9,-9.394,1,0.0399,0.624000,0.000000,0.1110,0.3400,87.115,4,210789,Fresh Out The Slammer,65,Taylor Swift
7,0.533,0.57300,0,-7.123,1,0.1380,0.178000,0.000000,0.3090,0.3980,150.088,4,215463,Florida!!! (feat. Florence + The Machine),73,Taylor Swift
8,0.626,0.42800,2,-8.374,1,0.0261,0.607000,0.000000,0.0921,0.4870,94.950,3,254365,Guilty as Sin?,78,Taylor Swift
9,0.606,0.33800,0,-10.555,1,0.0480,0.315000,0.000000,0.1060,0.2380,119.880,4,334084,Who’s Afraid of Little Old Me?,75,Taylor Swift


Here I tidy up the merged lists for both of the ablums.

In [158]:
ts_merged_tidy = ts_merged.drop(columns = ['href','name', 'artist', 'is_local', 'is_playable', 'track_number', 'type_y', 'uri_y', 'album', 'artists', 'disc_number', 'analysis_url', 'duration_ms_x', 'uri_x', 'track_href', 'type_x', 'explicit', 'external_ids', 'type_x', 'id', 'external_urls'])
ut_merged_tidy = ut_merged.drop(columns = ['href', 'preview_url', 'name', 'artist', 'is_local', 'is_playable', 'track_number', 'type_y', 'uri_y', 'album', 'artists', 'disc_number', 'analysis_url', 'duration_ms_x', 'uri_x', 'track_href', 'type_x', 'explicit', 'external_ids', 'type_x', 'id', 'external_urls'])

Here I seperate my two albums of choice and use the sum feature to find the averages the values of the endpoints.

In [159]:
ts_merged_tidy.mean()

danceability             0.551312
energy                   0.452937
key                      4.312500
loudness                -9.721937
mode                     0.937500
speechiness              0.068238
acousticness             0.463269
instrumentalness         0.000256
liveness                 0.152150
valence                  0.308375
tempo                  119.574187
time_signature           3.687500
duration_ms_y       244731.562500
popularity              72.562500
dtype: float64

In [160]:
ut_merged_tidy.mean()

danceability            0.633600
energy                  0.424126
key                     6.200000
loudness              -14.925200
mode                    0.600000
speechiness             0.105245
acousticness            0.515798
instrumentalness        0.768760
liveness                0.104340
valence                 0.597540
tempo                 114.560000
time_signature          3.650000
duration_ms_y       58833.400000
popularity             43.250000
dtype: float64

Here we see all the data condensed into averages from the mean function. Now I can compare the averages to each other and evaluate my hypothesis.
Data compared:
Danceability 
    Taylor: .55 
    Undertale: .63
Energy
    Taylor: .45
    Undertale: .42
Key
    Taylor: 4.31
    Undetale: 6.2
Loudness
    Taylor: -9.72
    Undertale: -14.93
mode
    Taylor: 0.94
    Undertale: 0.6
speechiness
    Taylor: .07
    Undertale: .11
acousticness
    Taylor: .46
    Undertale: .52
instrumentalness
    Taylor: .0003
    Undertale: .77
liveness
    Taylor: .152
    Undertale: .104
valance
    Taylor: .31
    Undertale: .6
tempo
    Taylor: 119.57
    Undertale: 114.56
time
    Taylor: 3.69
    Undertale: 3.65
Duration
    Taylor:  244731.6
    Undertale: 58833.4
Popularity 
    Taylor:  72.56
    Undertale: 43.25

Now with the data above in mind, we can draw some conclusions. There is a slight difference in mixing and style for video game music compared to popular music. If we look at the instrumentalness feature we will see one of the larger differences in this data. Undertale's soundtrack is vastly more instrumental with a .77 compared to an extremely small .0003 from Taylor Swift. Another large difference is the Loudness of the tracks. On average the Undertale soundtrack is 5 decibels quieter compared to Taylor Swift's value. The other large difference is the average duration of tracks. On average Undertale is 185898.2 milliseconds shorter compared to Taylor Swift's value. This probably comes from the common practice of many video game songs being loops that are around thirty seconds to a minute. 

Surprisingly the rest of the data is comparable and close to each other in value speechiness, tempo, energy, danceability, and mode are very close in value. The difference mainly comes from mixing volumes, popularity, words sung, and duration. So, therefore there are differences, but these differences are not as great as we think.

I think the next step would be to compare more albums and soundtracks to each other and then begin combining the respective data. Therefore you have a more holistic view of the comparison and a solid foundation to build more comparisons off of. Off of this foundation, you could create a comparative database to compare various popular albums and videogame soundtracks. 